In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as smt

from sklearn.feature_selection import VarianceThreshold 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import xgboost as XGB
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
df_train.head()

In [4]:
print('Shape of train set: {}'.format(df_train.shape))
print('Shape of test set: {}'.format(df_test.shape))

In [5]:
#do a cross check between train and test sets to verify all columns are present in both sets, except for the target variable in train set.

variables_not_in_train_set = [i for i in df_train.columns if i not in df_test.columns]
print('Columns not in test set but present in train set: {}'.format(variables_not_in_train_set))

In [6]:
df_train.info()

We can see quite a number of columns with missing values. We will deal with that later. 

In [7]:
#drop the Id column for both train and test sets
df_train.drop(columns=['Id'], inplace=True)

# Save the 'Id' list before dropping it from the test set
test_Id_list = df_test["Id"].tolist()
df_test.drop(columns=['Id'], inplace=True)

# 1. Numerical Data 

We first deal with the data by grouping them into numerical and categorical data. We start off with the numerical data and perform the following investigations:

* Determine the Pearson Correlation coefficient of each features with the target variable. We will drop any features which are deemed poorly correlated with the target variable.

* Drop any features with multicollinearity.

* Drop any features with low variance.

* Determine the distribution of all selected features.
 
* Impute any missing values or drop any features when imputation doesnt help. 


In [8]:
#Filter out a list containing only numerical variables
df_train_numerical = [i for i in df_train.columns if df_train[i].dtype != 'object']
print('Numerical variables: ', df_train_numerical)

#compute the correlation heatmap to see which features are highly correlated with target
corr_matrix = df_train[df_train_numerical].corr()
plt.figure(figsize=(20,20), dpi=70)
sns.heatmap(corr_matrix, cmap=plt.cm.Reds, annot=True)
plt.show()

In [9]:
corr_with_SalePrice = df_train[df_train_numerical].corr()['SalePrice'][:-1]
print('Features highly correlated with SalePrice: ', corr_with_SalePrice[abs(corr_with_SalePrice) >= 0.5].sort_values(ascending=False).round(2))

In [10]:
print('Features fairly correlated with SalePrice: ', corr_with_SalePrice[(abs(corr_with_SalePrice) < 0.5) & (abs(corr_with_SalePrice) > 0.3)].sort_values(ascending=False).round(2))

In [11]:
print('Features poorly correlated with SalePrice: ', corr_with_SalePrice[abs(corr_with_SalePrice) < 0.3].sort_values().round(2))

Features that have a pearson correlation coefficient of less than 0.30 will be dropped, as they have very little to no linear relationship with our target variable. 

In [12]:
#strong features with corr more than 0.5
strong_features = corr_with_SalePrice[corr_with_SalePrice >= 0.5].sort_values(ascending=False).index
strong_list = [x for x in strong_features]
strong_list.append('SalePrice')

def reg_plot(df, features, rows, col):
    fig = plt.figure(figsize=(19,19), dpi=70)
    for i, feature in enumerate(features):
        if feature != 'SalePrice':
            ax = fig.add_subplot(rows, col, i+1)
            sns.regplot(x=feature, y='SalePrice', data=df, line_kws={'color':'black'})
            ax.set_xlabel(feature)
            ax.set_ylabel('SalePrice')
    
reg_plot(df_train[strong_list], strong_list, 4, 3)

In [13]:
#features with fair corr between 0.3 to 0.5
fair_features = corr_with_SalePrice[(corr_with_SalePrice < 0.5) & (corr_with_SalePrice > 0.3)].sort_values(ascending=False).index
fair_list = [x for x in fair_features]
fair_list.append('SalePrice')

reg_plot(df_train[fair_list], fair_list, 4, 3)

In [14]:
#detecting multicollinearity amongst our numerical variables

rows, cols = df_train[df_train_numerical].shape
columns_list = list(df_train[df_train_numerical].columns)

df = []
corr = df_train[df_train_numerical].corr().values
for i in range(cols):
    for j in range(i+1, cols):
        if corr[i,j] > 0.7:
            df.append([columns_list[i], columns_list[j], corr[i,j].round(2)])

df1 = pd.DataFrame(df, columns=['Variable 1', 'Variable 2', 'Corr']).sort_values(by=['Corr'], ascending=False)
df1[df1['Variable 2'] != 'SalePrice']

From our correlation heatmap, we detected 4 pairs of features which are multicollinear, with correlation values of more than 0.80. As such, we will be dropping each of the features within each pairs.

In [15]:
#keeping variables with fair to strong corr with SalePrice
strong_corr = corr_with_SalePrice[abs(corr_with_SalePrice) >= 0.5].sort_values(ascending=False).round(2)
fair_corr = corr_with_SalePrice[(abs(corr_with_SalePrice) < 0.5) & (abs(corr_with_SalePrice) > 0.3)].sort_values(ascending=False).round(2)

features_to_keep = list(strong_corr.index) + list(fair_corr.index)

#drop variables with multicollinearity
numerical_to_drop = ['GarageCars', 'GarageYrBlt', 'TotRmsAbvGrd', '1stFlrSF']
numerical_to_keep = [x for x in features_to_keep if x not in numerical_to_drop]
numerical_to_keep.append('SalePrice')

print('Numerical features to be kept:')
print(numerical_to_keep)

In [16]:
#updated train set after dropping
df_train1_numerical = df_train[numerical_to_keep]

#updated test set after dropping
numerical_to_keep.remove('SalePrice')
df_test1_numerical = df_test[numerical_to_keep]

#check to ensure columns are dropped
print(df_train1_numerical.shape)
print(df_test1_numerical.shape)

In [17]:
#cross check to all columns are present in both dataset
variables_not_in_train_set = [i for i in df_train1_numerical.columns if i not in df_test1_numerical.columns]
print('Columns not in test set but present in train set: ', variables_not_in_train_set)

After dropping features, we double check again to ensure the number of columns and rows for both train and test sets are tallied. 

Next, we check for any features with small variations (low variance). These features are likely to have small impact on the final price of the house. Keeping these features is of no use to our prediction model. As such, we will drop these features, if any. We set the threshold at 5%, meaning that any features with 95% or more similar values will be dropped.

In [18]:
#removing quasi-features that have low variance

var = VarianceThreshold(threshold = 0.05) 
var.fit(df_train1_numerical)

col_to_drop = [x for x in df_train1_numerical.columns if x not in df_train1_numerical.columns[var.get_support()]]
print('Columns with low variance to be dropped: ', col_to_drop)

In [19]:
#have a look at the overall distributions of our remaining numerical variables

df_train1_numerical.hist(bins=50, figsize=(15,15))
plt.show()

From the distributions, we actually notice a mixture of both discrete and continuous features. 

# 1.1 Missing Values

We first with missing values from the train set.


In [20]:
#filter out columns with missing values and calculate its proportion compared to total number of data

col_missing_values_train = [x for x in df_train1_numerical.columns if df_train1_numerical[x].isnull().any()]
col_df_missing_train = pd.DataFrame(df_train1_numerical[col_missing_values_train].isnull().sum(), columns=['No. of Missing Values'])
col_df_missing_train['Percent of total data'] = (100 * col_df_missing_train['No. of Missing Values'] / df_train1_numerical.shape[0]).round(2)

print('Numerical columns with missing values: ', col_missing_values_train)
plt.figure(figsize=(8,8), dpi=70)
sns.set_style('darkgrid')
sns.barplot(x=col_df_missing_train.index, y=col_df_missing_train['Percent of total data'], data=col_df_missing_train)
plt.title('% of missing values')
plt.show()

The missing values in 'LotFrontage' makes up more than 17.5% of the total number of rows. 

We will next proceed to impute missing values for both features with their respective median values. 

In [21]:
simple_imp = SimpleImputer(strategy = 'median')

imputed_df_numerical_train = pd.DataFrame(simple_imp.fit_transform(df_train1_numerical))
imputed_df_numerical_train.columns = df_train1_numerical.columns

In [22]:
sns.set(rc={'figure.figsize':(14,12)})
fig, ax = plt.subplots(2, 2)

for pos, feature in enumerate(col_missing_values_train):
    sns.histplot(df_train1_numerical[feature], bins=30, kde=True, ax = ax[pos, 0])
    ax[pos, 0].set_ylabel('Before Imputation')
    
    sns.histplot(imputed_df_numerical_train[feature], bins=30, kde=True, ax = ax[pos, 1])
    ax[pos, 1].set_ylabel('After Imputation')

plt.show()

We check the distribution of the two features to see if there is any major change or shift to the distributions. 'LotFrontage' shows a significant change in distribution (as shown by the shift in its mode). As such, we will drop the feature 'LotFrontage' and keep 'MasVnrArea'.

In [23]:
#dropping the feature 'LotFrontage' from our train set

imputed_df_numerical_train.drop(['LotFrontage'], axis=1, inplace=True)

imputed_df_numerical_train.head()

To account equally for both train and test sets, we will also drop the feature 'LotFrontage' from our test set.

In [24]:
#dropping the feature 'LotFrontage' from our test set

df_test1_numerical.drop(['LotFrontage'], axis=1, inplace=True)

df_test1_numerical.head()

Next, we deal with missing values from the test set.

In [25]:
col_missing_values_test = [x for x in df_test1_numerical.columns if df_test1_numerical[x].isnull().any()]
col_df_missing_test = pd.DataFrame(df_test1_numerical[col_missing_values_test].isnull().sum(), columns=['No. of Missing Values'])
col_df_missing_test['Percent of total data'] = (100 * col_df_missing_test['No. of Missing Values'] / df_test1_numerical.shape[0]).round(2)

print('Numerical columns with missing values: ', col_missing_values_test)
plt.figure(figsize=(8,8), dpi=70)
sns.set_style('darkgrid')
sns.barplot(x=col_df_missing_test.index, y=col_df_missing_test['Percent of total data'], data=col_df_missing_test)
plt.title('% of missing values')
plt.show()

The missing values in 'MasVnrArea' makes up more than 1% of the total number of rows, while the rest make up less than 0.2% of the total. 

We will next proceed to impute missing values for all features with their respective median values.

In [26]:
#impute the missing columns with median values

simple_imp = SimpleImputer(strategy = 'median')

imputed_df_numerical_test = pd.DataFrame(simple_imp.fit_transform(df_test1_numerical))
imputed_df_numerical_test.columns = df_test1_numerical.columns

In [27]:
sns.set(rc={'figure.figsize':(17,15)})
fig, ax = plt.subplots(4, 2)

for pos, feature in enumerate(col_missing_values_test):
    sns.histplot(df_test1_numerical[feature], bins=30, kde=True, ax=ax[pos, 0])
    ax[pos, 0].set_ylabel('Before Imputation')
    
    sns.histplot(imputed_df_numerical_test[feature], bins=30, kde=True, ax=ax[pos, 1])
    ax[pos, 1].set_ylabel('After Imputation')
    
plt.show()

There are no significant changes to the distribution of our features above after imputation. Hence, we shall keep all of them.

In [28]:
#check to ensure all columns have no missing values for both data sets

pd.DataFrame([imputed_df_numerical_train.isnull().sum(), imputed_df_numerical_test.isnull().sum()], index=['Train', 'Test'])

Double check both data sets again to ensure no more missing values in all columns

In [29]:
print('Final number of numerical variables in train set: ', imputed_df_numerical_train.shape)
print('Final number of numerical variables in test set: ', imputed_df_numerical_test.shape)

In [30]:
#cross check to all columns are present in both dataset

variables_not_in_train_set = [i for i in imputed_df_numerical_train.columns if i not in imputed_df_numerical_test.columns]
print('Columns not in test set but present in train set: ', variables_not_in_train_set)

variables_not_in_test_set = [i for i in imputed_df_numerical_test.columns if i not in imputed_df_numerical_train.columns]
print('Columns not in test set but present in train set: ', variables_not_in_test_set)

A cross check of both our train and test sets again shows all columns are the same and properly accounted for. 

# 2. Categorical data

Next up, we will deal with the categorical data by performing the following investigations:

* Determine the distribution of all categorical features with bar charts.

* Drop any features with low variance.
 
* Impute any missing values or drop any features when imputation doesnt help. 

* Determine the relationship between the categorical features and our target variable with box plots.

In [31]:
df_categorical = [i for i in df_train.columns if df_train[i].dtype == 'object']

print(f'Categorical variables: {df_categorical}\n')
print(f'Number of categorical variables: {len(df_categorical)}\n')

In [32]:
#define our train and test df, only include categorical variables

df_train_categorical = pd.concat([df_train[df_categorical], df_train['SalePrice']], axis=1)
df_test_categorical = df_test[df_categorical]

Next, we find out the distribution of our variables.

In [33]:
#overview of distributions with bar charts

fig, ax = plt.subplots(15, 3, figsize=(15, 40))

for i, ax in enumerate(fig.axes):
    if i < len(df_categorical):
        sns.countplot(x=df_categorical[i], data=df_train[df_categorical], ax=ax)
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=90)
        ax.set_xlabel(df_categorical[i])

fig.tight_layout()
plt.show()

Through eyeballing, we can detect a few variables that are dominated by a single group. We can remove these features as they have little to no impact on the prediction of house prices. 

In [34]:
catcol_to_be_dropped = ['Street', 
                        'LandContour', 
                        'Utilities', 
                        'LandSlope', 
                        'Condition2', 
                        'RoofMatl', 
                        'BsmtCond', 
                        'BsmtFinType2',
                        'Heating', 
                        'CentralAir', 
                        'Electrical', 
                        'Functional', 
                        'GarageQual', 
                        'GarageCond',
                        'PavedDrive']

In [35]:
#drop the columns for both train and test sets

df_train_categorical.drop(catcol_to_be_dropped, axis=1, inplace=True)
df_test_categorical.drop(catcol_to_be_dropped, axis=1, inplace=True)

In [36]:
#check to see if columns are dropped

print(f'Train set: {df_train_categorical.shape}\n')
print(f'Test set: {df_test_categorical.shape}\n')

We will next visualize how the features vary with our target variable with box plots.

In [37]:
#overview of features vs target variable

fig, ax = plt.subplots(10, 3, figsize=(15, 40))

for i, ax in enumerate(fig.axes):
    if i < len(df_train_categorical.columns)-1:
        sns.boxplot(x=df_train_categorical.columns[i], y='SalePrice', data=df_train_categorical, ax=ax)
        ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation = 90)
        
fig.tight_layout()
plt.show()
        

From the boxplots, we notice some variables have similar pattern of group distributions, which might suggest co-dependency between each pairs. The following pairs of variables have very similar distribution:

* 'ExterQual' and 'BsmtQual' (pair_1)
* 'MasVnrType' and 'ExterQual' (pair_2)
* 'Exterior1st' and 'Exterior2nd' (pair_3)
* 'GarageType' and 'SaleCondition' (pair_4)
* 'MasVnrType' and 'BsmtQual' (pair_5)
* 'BsmtQual' and 'KitchenQual' (pair_6)

we shall perform a Chi-squared test of independence for each pair of variables at 5% significance level to determine whether or not they have any strong dependency. For any pair with strong dependency, we will drop one of variable within the pair.  

In [38]:
#loop over each pair and crosstab all of them

v1 = ['ExterQual', 'MasVnrType', 'Exterior1st', 'GarageType', 'MasVnrType', 'BsmtQual']
v2 = ['BsmtQual', 'ExterQual', 'Exterior2nd', 'SaleCondition', 'BsmtQual', 'KitchenQual']

crosstab_arrays = []
for i, j in zip(v1, v2):
    x = np.array(pd.crosstab(df_train_categorical[i], df_train_categorical[j]))
    crosstab_arrays.append(x)

To simplify the crosstabbing process, we loop over the 6 pairs and assign their crosstabs to a list. Subsequently, we will proceed to conduct our Chi_squared test for each pair separately. 

We set the significance level for our test to be at 5%. We set the hypotheses as follow:

**Null hypothesis (H0)**: Variables are independent

**Alternative hypothesis (H1)**: Variables are dependent

In [39]:
#evaluate p-values of chi-squared test

for i, j in enumerate(zip(v1, v2)):
    chi2sq_result = stats.chi2_contingency(crosstab_arrays[i])
    print(f"p-value of Chi-squared test bewteen {j[0]} and {j[1]} is: {chi2sq_result[1]}")
    if chi2sq_result[1] < 0.05:
        print(f'Variables are dependent (Reject H0)\n')
    else:
        print(f'Variables are independent (Fail to reject H0)\n')

For all tests, we reject H0 and conclude that the variables are all dependent. As such, we have to drop one of the co-dependent variables for all 6 pairs.

In [40]:
df_train_categorical.drop(v1, axis=1, inplace=True)
df_test_categorical.drop(v1, axis=1, inplace=True)

In [41]:
#check to see if columns are dropped

print(f'Train set: {df_train_categorical.shape}\n')
print(f'Test set: {df_test_categorical.shape}\n')

# 2.1 Missing Values

We will now investigate all categorical columns with missing values and impute, or drop them if necessary.

In [42]:
#visualize train set columns with missing values

df_missing_categorical_train = pd.DataFrame(100 * df_train_categorical.isnull().sum() / df_train_categorical.shape[0], columns=['Percent of total data']).sort_values(by='Percent of total data', ascending=False)

df_filtered_missing_train = df_missing_categorical_train[df_missing_categorical_train['Percent of total data'] > 0]

sns.barplot(y=df_filtered_missing_train.index, x='Percent of total data', orient='h', data=df_filtered_missing_train)
plt.title('Missing data for Categorical Variables (Train set)', fontsize=18)
plt.show()

We shall drop any variable with missing values comprising more than 30% of its total data.  

In [43]:
NaN_col_to_drop_train = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']

#drop the variables in both train and test set

df_train_categorical.drop(NaN_col_to_drop_train, axis=1, inplace=True)
df_test_categorical.drop(NaN_col_to_drop_train, axis=1, inplace=True)

For the remaining variables with less than 5% of missing values, we shall impute all missing values with values from their corresponding modal classes. 

In [44]:
#impute our missing values in train set with mode

NaN_col_to_impute = {'GarageFinish': df_train_categorical['GarageFinish'].mode().iloc[0],
 'BsmtExposure': df_train_categorical['BsmtExposure'].mode().iloc[0],
 'BsmtFinType1': df_train_categorical['BsmtFinType1'].mode().iloc[0]}

imputed_df_train_categorical = df_train_categorical.fillna(value=NaN_col_to_impute)

Similarly, we also check for variables with missing values in our test set, and then impute or drop them if necessary.

In [45]:
#visualize test set columns with missing values

df_missing_categorical_test = pd.DataFrame(100 * df_test_categorical.isnull().sum() / df_test_categorical.shape[0], columns=['Percent of total data']).sort_values(by='Percent of total data', ascending=False)

df_filtered_missing_test = df_missing_categorical_test[df_missing_categorical_test['Percent of total data'] > 0]

sns.barplot(y=df_filtered_missing_test.index, x='Percent of total data', orient='h', data=df_filtered_missing_test)
plt.title('Missing data for Categorical Variables (Test set)', fontsize=18)
plt.show()

Most of the missing values for our test set columns are less than 5%. As such, we will impute all missing values with their corresponding modal classes. 

In [46]:
#impute our missing values in test set with mode

NaN_col_to_impute = {'GarageFinish': df_test_categorical['GarageFinish'].mode().iloc[0],
                     'BsmtExposure': df_test_categorical['BsmtExposure'].mode().iloc[0],
                     'BsmtFinType1': df_test_categorical['BsmtFinType1'].mode().iloc[0],
                     'MSZoning': df_test_categorical['MSZoning'].mode().iloc[0],
                     'SaleType': df_test_categorical['SaleType'].mode().iloc[0],
                     'KitchenQual': df_test_categorical['KitchenQual'].mode().iloc[0],
                     'Exterior2nd': df_test_categorical['Exterior2nd'].mode().iloc[0]}

imputed_df_test_categorical = df_test_categorical.fillna(value=NaN_col_to_impute)

In [47]:
#finally, check to ensure there are no more categorical columns with missing values in train and test set

pd.DataFrame([imputed_df_train_categorical.isnull().sum(), imputed_df_test_categorical.isnull().sum()], index=['Train', 'Test'])

In [48]:
#final check to see if columns are dropped and tallied between train and test set

print(f'Train set: {df_train_categorical.shape}\n')
print(f'Test set: {df_test_categorical.shape}\n')

**Encoding Categorical Variables**

Before we can use the data for training, we have to transform them into numerical values. This will be done by using one-hot encoding.

In [49]:
#transform our train set with pandas function get_dummies()

df_train_categorical.drop(['SalePrice'], axis=1, inplace=True)

df_train_categorical_dummies = pd.get_dummies(df_train_categorical, drop_first=True)
df_train_categorical_dummies.head()

In [50]:
#transform our test set with pandas function get_dummies()

df_test_categorical_dummies = pd.get_dummies(df_test_categorical, drop_first=True)
df_test_categorical_dummies.head()

In [51]:
#check to see which variables are not present in either sets after encoding

variables_not_in_test_set = [i for i in df_train_categorical_dummies.columns if i not in df_test_categorical_dummies.columns]
variables_not_in_train_set = [i for i in df_test_categorical_dummies.columns if i not in df_train_categorical_dummies.columns]
print('Columns not in train set but present in test set: ', variables_not_in_test_set)
print('Columns not in test set but present in train set: ', variables_not_in_train_set)

'HouseStyle_2.5Fin', 'Exterior2nd_Other' are the two dummy variables not present in the test set. Hence, we will drop these two columns in our train set.

In [52]:
#dropping the two dummy variables and check again

df_train_categorical_dummies.drop(['HouseStyle_2.5Fin', 'Exterior2nd_Other'], axis=1, inplace=True)

variables_not_in_test_set = [i for i in df_train_categorical_dummies.columns if i not in df_test_categorical_dummies.columns]
print('Columns not in test set but present in train set: ', variables_not_in_train_set)

In [53]:
print(f'Train set: {df_train_categorical_dummies.shape}\n')
print(f'Test set: {df_test_categorical_dummies.shape}\n')

**Merge both Numerical and Categorical data together as final data**

In [54]:
new_train_set = pd.concat([imputed_df_numerical_train, df_train_categorical_dummies], axis=1)
print(f'Shape of new train set: {new_train_set.shape}\n')

new_test_set = pd.concat([imputed_df_numerical_test, df_test_categorical_dummies], axis=1)
print(f'Shape of new test set: {new_test_set.shape}\n')

# 3. Feature Transformation

Right at the start of our analysis, some numerical features were spotted to be skewed. To ensure a good accuracy for our model, we need to transform these features such that their distributions are closer to a normal distribution. Ideally, a normally distributed data allows the machine learning model to learn from a wider range of data, rather than learning data that are skewed and ending up with a bias model. 

Before we proceed to apply data transformations, we need to convert 'YearBuilt' and 'YearRemodAdd' to age of the house since it was first built, and age of the house since it was first remodelled. It does not make sense to transform 'YearBuilt' and 'YearRemodAdd' as Year is an ordinal feature. As such, they will both be removed after conversion to age.  

In [55]:
#converting to age by creating new column 'Age Since Built'

new_train_set['Age Since Built'] = new_train_set['YearBuilt'].max() - new_train_set['YearBuilt']
new_test_set['Age Since Built'] = new_test_set['YearBuilt'].max() - new_test_set['YearBuilt']

#drop 'YearBuilt'

new_train_set.drop(["YearBuilt"], axis=1, inplace=True)
new_test_set.drop(["YearBuilt"], axis=1, inplace=True)

In [56]:
#converting to age by creating new column 'Age Since Remod'

new_train_set['Age Since Remod'] = new_train_set['YearRemodAdd'].max() - new_train_set['YearRemodAdd']
new_test_set['Age Since Remod'] = new_test_set['YearRemodAdd'].max() - new_test_set['YearRemodAdd']

#drop 'YearBuilt'

new_train_set.drop(['YearRemodAdd'], axis=1, inplace=True)
new_test_set.drop(['YearRemodAdd'], axis=1, inplace=True)

We will now compute the skewness of our features, and then proceed to filter out features which are considered to be too skewed.

We then apply the log transformation technique to the skewed features.

In [57]:
#numerical features

numerical_features = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', 'FullBath', 'Age Since Built', 'Age Since Remod', 'MasVnrArea', 'Fireplaces', 'BsmtFinSF1', 'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'SalePrice']

#applying log transformation to the skewed features for both train and test sets

skewed_features = [x for x in numerical_features if new_train_set[x].skew() >= 0.5]

for i in skewed_features:
    if i != 'SalePrice':
        new_train_set[i] = np.log((new_train_set[i])+1)
        new_test_set[i] = np.log((new_test_set[i])+1)
    else:
        new_train_set[i] = np.log((new_train_set[i])+1)

In [58]:
#compute the skewness of all numerical features after transformation

print(f'Feature Skewness after Transformation:\n')
print(new_train_set[numerical_features].skew())

#check the feature distributions after transformation

new_train_set[numerical_features].hist(bins=50, figsize=(15,15))
plt.show()

Judging from the computed values of skewness and the distributions above, we can see that most of the features that were previously skewed, are now more normally distributed. 

In [59]:
#check the scatter plots (against SalePrice) of our transformed features

reg_plot(new_train_set[numerical_features], numerical_features, 5, 3)

From the scatter plots above, we notice previously heteroskedastic variables like 'GrLivArea' and 'TotalBsmtSF' are now more homoskedastic after transformation. As the transformation has improved the quality of our data, they are now ready to be used for modelling.

**Splitting Data into Training and Testing sets**

In [60]:
#define X and y
X = new_train_set.drop(['SalePrice'], axis=1)

y = new_train_set.loc[:, 'SalePrice']

#splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}\n')
print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')

# 4. Predictive Modelling

# 4.1 Multiple Linear Regression

We will be using the backward elimination here, in which all features will be fed to the model first. We check the performance of the model and then iteratively remove the worst performing features one by one till the overall performance of the model comes in acceptable range. The performance metric used here to evaluate feature performance is p-value. If the p-value of the feature is more than 0.05, we remove the feature. We also do the same for other features.

In [ ]:
#Iterative feature selection using Backward Elimination

cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = smt.add_constant(X_1)
    model = smt.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
model_LinR = LinearRegression()
model_LinR.fit(X_train[selected_features_BE], y_train)
y_LinR_pred = model_LinR.predict(X_test[selected_features_BE])

print( 'R2_Score: ', r2_score(y_test, y_LinR_pred))
print("RMSE: ", mean_squared_error(y_test, y_LinR_pred, squared=False))

# 4.2 Ridge Regression

In [ ]:
alphas = np.linspace(0, 10, 100).tolist()

tuned_parameters = {"alpha": alphas}
ridge_cv = GridSearchCV(Ridge(), tuned_parameters, cv=10, n_jobs=-1, verbose=1)

ridge_cv.fit(X_train[selected_features_BE], y_train)

# print best params and the corresponding R²
print(f"Best hyperparameters: {ridge_cv.best_params_}")
print(f"Best R² (train): {ridge_cv.best_score_}")

In [ ]:
model_ridge_opt = Ridge(alpha = ridge_cv.best_params_["alpha"])
model_ridge_opt.fit(X_train[selected_features_BE], y_train)
y_pred_ridge_opt = model_ridge_opt.predict(X_test[selected_features_BE])
acc_ridge_opt = model_ridge_opt.score(X_test[selected_features_BE], y_test)

print( 'R2_Score: ', acc_ridge_opt)
print("RMSE: ", mean_squared_error(np.exp(y_test), np.exp(y_pred_ridge_opt), squared=False))

# 4.3 Lasso Regression

In [ ]:
alphas = np.linspace(0, 10, 100).tolist()

tuned_parameters = {"alpha": alphas}
lasso_cv = GridSearchCV(Lasso(), tuned_parameters, cv=10, n_jobs=-1, verbose=1)

lasso_cv.fit(X_train[selected_features_BE], y_train)

# print best params and the corresponding R²
print(f"Best hyperparameters: {lasso_cv.best_params_}")
print(f"Best R² (train): {lasso_cv.best_score_}")

An optimal value of 0 for its alpha value means the Lasso regression may not be the most ideal model in this case. We shall set the value for alpha to be 0.001.

In [ ]:
model_lasso = Lasso(alpha = 0.001)
pipe = make_pipeline(StandardScaler(), model_lasso)
pipe.fit(X_train[selected_features_BE], y_train)
y_pred_ridge = pipe.predict(X_test[selected_features_BE])
acc_ridge = pipe.score(X_test[selected_features_BE], y_test)

print( 'R2_Score: ', acc_ridge)
print("RMSE: ", mean_squared_error(np.exp(y_test), np.exp(y_pred_ridge), squared=False))

# 4.4 XGBoost Regressor

In [ ]:
# Define hyperparameters
tuned_parameters_xgb = {"max_depth": [3],
                        "colsample_bytree": [0.3, 0.7],
                        "learning_rate": [0.01, 0.05, 0.1],
                        "n_estimators": [100, 500, 1000]}

# GridSearch
xgbr_cv = GridSearchCV(estimator=XGBRegressor(),
                       param_grid=tuned_parameters_xgb,
                       cv=5,
                       n_jobs=-1,
                       verbose=1)

# fit the GridSearch on train set
xgbr_cv.fit(X_train[selected_features_BE], y_train)

# print best params and the corresponding R²
print(f"Best hyperparameters: {xgbr_cv.best_params_}\n")
print(f"Best R²: {xgbr_cv.best_score_}")

In [ ]:
model_xgb_opt = XGBRegressor(colsample_bytree = xgbr_cv.best_params_["colsample_bytree"],
                             learning_rate = xgbr_cv.best_params_["learning_rate"],
                             max_depth = xgbr_cv.best_params_["max_depth"],
                             n_estimators = xgbr_cv.best_params_["n_estimators"])

model_xgb_opt.fit(X_train[selected_features_BE], y_train)
y_pred_xgb_opt = model_xgb_opt.predict(X_test[selected_features_BE])
acc_xgb_opt = model_xgb_opt.score(X_test[selected_features_BE], y_test)

print( 'R2_Score: ', acc_xgb_opt)
print("RMSE: ", mean_squared_error(np.exp(y_test), np.exp(y_pred_xgb_opt), squared=False))

# 5. Prediction Using the Best Model

In [ ]:
y_pred_with_bestmodel = np.exp(model_LinR.predict(new_test_set[selected_features_BE]))

output = pd.DataFrame({"Id": test_Id_list,
                       "SalePrice": y_pred_with_bestmodel})

output.head(5)

In [ ]:
# Save the output
output.to_csv("submission.csv", index=False)